### Importing necessary libraries

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

%matplotlib inline

### Checking the directories of the datasets available

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Importing the dataset

In [ ]:
dataset = pd.read_csv("../input/santander-value-prediction-challenge/train.csv")
dataset.shape

### Printing the first five records

In [ ]:
dataset.head(5)

## Data Exploration / Исследование данных

### Statistics of the data / Statistics of the data

In [ ]:
dataset.describe()

### Checking the null values in the data

In [ ]:
dataset.isnull().sum()

### Checking the data type of the columns

In [ ]:
dataset.dtypes

### Listing all the datatypes used in the dataset / Перечисление всех типов данных, используемых в наборе данных

In [ ]:
dataset.dtypes.value_counts()

### Removing constant columns

In [ ]:
columns = dataset.std() == 0
const_columns = columns.iloc[[i for i, x in enumerate(columns) if x]]
dataset.drop(const_columns.index, axis = 1, inplace = True)

### Removing sparse columns / разбросанные

In [ ]:
def drop_sparse(train):
    sparse_columns = []
    flist = dataset.columns[2:]
    for f in flist:
        if len(np.unique(train[f]))<2:
            sparse_columns.append(f)
    return sparse_columns

sparse_columns = drop_sparse(dataset)
dataset.drop(sparse_columns, axis = 1, inplace = True)

### Removing duplicate columns

In [ ]:
def duplicate_columns(dataset):
    groups = dataset.columns.to_series().groupby(dataset.dtypes).groups
    my_dict = {}
    duplicate_features = []
    
    for d_type, columns in groups.items():
        columns_group = dataset[columns]
        list_of_column_names = dataset[columns].columns
        length = len(columns)
        
        for i in range(length):
            a = tuple(columns_group.iloc[:, i])
            if a in my_dict:
                duplicate_features.append(list_of_column_names[i])
            else:
                my_dict[a] = list_of_column_names[i]
            
    return duplicate_features

duplicate_features = duplicate_columns(dataset)
dataset.drop(sparse_columns, axis = 1, inplace = True)

## Model Training

### XGBoost

In [ ]:
dataset = dataset.drop(['ID'], axis = 1)
X = dataset.iloc[:,1:]       
Y = dataset.iloc[:,0]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.21, random_state=42)

In [ ]:
xb_model = XGBRegressor(max_depth=6, n_estimators=80, learning_rate=0.3)
'''xb_model = XGBRegressor(colsample_bytree=0.055, colsample_bylevel =0.5, 
                             gamma=1.5, learning_rate=0.01, max_depth=32, 
                             objective='reg:linear',booster='gbtree',
                             min_child_weight=57, n_estimators=800, reg_alpha=0, 
                             reg_lambda = 0,eval_metric = 'rmse', subsample=0.7, 
                             silent=1, n_jobs = -1, early_stopping_rounds = 14,
                             random_state =7, nthread = -1)'''
xb_model.fit(x_train, y_train,
             eval_set=[(x_test, y_test)], verbose=False)

In [ ]:
# Predicting the values form the test_set / Прогнозирование значений из набора test_set
y_pred = xb_model.predict(x_test)

# the root-mean squared error for predictions / среднеквадратичная ошибка для прогнозов
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

In [ ]:
# Comparing the predictions rmse with benchmark rmse / Сравнение прогнозы rmse с тестом rmse
y_mean = [y_test.mean()] * y_test.shape[0]

rmse_benchmarch = np.sqrt(mean_squared_error(y_mean, y_pred))
print(rmse_benchmarch)

In [ ]:
# Loading the test dataset for predicting the target feature / Загрузка набора тестовых данных для прогнозирования целевого объекта
test_dataset = pd.read_csv("../input/santander-value-prediction-challenge/test.csv")
test_id_s = test_dataset['ID']
test_dataset = test_dataset[dataset.columns[1:]]
test_pred_xgb = xb_model.predict(test_dataset)
test_pred_xgb = np.clip(test_pred_xgb, 1, float('inf'))

In [ ]:
final_preds = (test_pred_xgb)
pred_df = pd.DataFrame({'ID':test_id_s, 'target': final_preds})
pred_df.to_csv("submission99.csv", float_format="%.10g", index = False)